# Not Possible to create a multiclass model for the whole

In [0]:
import statsmodels.api as st
import pandas as pd
import numpy as np

In [0]:
'''Loading Data from selected mortality dataset(only 15 columns)'''
df=[] #len=11, each = 1yr
for i in range(2005,2016):
    url = 'data/mortality_selected/'+str(i)+'.csv'
    df1 = pd.read_csv(url)
    df.append(df1)
    print('Loading data of year',i)
print('Finish Loading!')
del df1
display(df[0][:1])


Loading data of year 2005
Loading data of year 2006
Loading data of year 2007
Loading data of year 2008
Loading data of year 2009
Loading data of year 2010
Loading data of year 2011
Loading data of year 2012
Loading data of year 2013
Loading data of year 2014
Loading data of year 2015
Finish Loading!


,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age,marital_status,day_of_week_of_death,current_data_year,injury_at_work,manner_of_death,activity_code,race,5y_age_band_recode,rankable_cause_recode
0,11.0,NaN,0,1,F,45,M,2,2005,U,7.0,NaN,1,10,4


In [0]:
"one hot encoding"
for i in range(11):
    sex_onehot = pd.get_dummies(df[i]['sex'], prefix='sex')
    injury_at_work_onehot = pd.get_dummies(df[i]['injury_at_work'], prefix='injury_at_work')
    df[i] = pd.concat([df[i], sex_onehot, injury_at_work_onehot], axis=1)
    df[i].drop(['sex', 'injury_at_work'], axis=1, inplace=True)
df[0]

,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,detail_age,marital_status,day_of_week_of_death,current_data_year,manner_of_death,activity_code,race,5y_age_band_recode,rankable_cause_recode,sex_F,sex_M,injury_at_work_N,injury_at_work_U,injury_at_work_Y
0,11.0,NaN,0,1,45,M,2,2005,7.0,NaN,1,10,4,1,0,0,1,0
1,13.0,NaN,0,1,61,D,7,2005,7.0,NaN,1,13,13,0,1,0,1,0
2,12.0,NaN,0,1,79,D,1,2005,7.0,NaN,1,16,9,1,0,0,1,0
3,12.0,NaN,0,1,50,S,4,2005,7.0,NaN,1,11,5,0,1,0,1,0
4,14.0,NaN,0,1,68,M,2,2005,7.0,NaN,1,14,4,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452501,NaN,9.0,1,12,59,U,1,2005,NaN,NaN,1,12,7,0,1,0,1,0
2452502,NaN,1.0,1,9,2,S,4,2005,NaN,NaN,2,1,18,0,1,0,1,0
2452503,NaN,1.0,1,8,22,S,4,2005,NaN,NaN,2,1,18,1,0,0,1,0
2452504,NaN,1.0,1,8,1,S,3,2005,NaN,NaN,1,1,18,0,1,0,1,0


In [0]:
'''Create X'''
used_cols = ['current_data_year', 'month_of_death', 'sex_F', 'sex_M', 'race', 'injury_at_work_N', 'injury_at_work_U', 'injury_at_work_Y', '5y_age_band_recode']
unused_cols = ['detail_age','education_1989_revision', 'education_2003_revision', 'education_reporting_flag','marital_status', 'day_of_week_of_death','manner_of_death', 'activity_code']
for i in range(11):
    df[i].drop(unused_cols,axis=1,inplace=True)
    if i==0:
        X = df[0]
    else:
        X = pd.concat([X,df[i]])
X = X[X['rankable_cause_recode'].isin([7,4,22,13,9,24,6,5])]
X

,month_of_death,current_data_year,race,5y_age_band_recode,rankable_cause_recode,sex_F,sex_M,injury_at_work_N,injury_at_work_U,injury_at_work_Y
0,1,2005,1,10,4,1,0,0,1,0
1,1,2005,1,13,13,0,1,0,1,0
2,1,2005,1,16,9,1,0,0,1,0
3,1,2005,1,11,5,0,1,0,1,0
4,1,2005,1,14,4,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2718192,10,2015,2,10,22,1,0,0,1,0
2718194,12,2015,1,10,7,1,0,0,1,0
2718195,12,2015,1,7,22,0,1,0,1,0
2718196,7,2015,2,17,7,0,1,0,1,0


In [0]:
url = 'data/generated data/multiclass_model_data.csv'
X.to_csv(url, index=False, sep=',')

In [0]:
Y = X['rankable_cause_recode']
Y
X.drop(['rankable_cause_recode'], axis=1, inplace=True)

In [0]:
Y

0           4
1          13
2           9
3           5
4           4
           ..
2718192    22
2718194     7
2718195    22
2718196     7
2718197     7
Name: rankable_cause_recode, Length: 23721259, dtype: int64

In [0]:
X

,month_of_death,current_data_year,race,5y_age_band_recode,sex_F,sex_M,injury_at_work_N,injury_at_work_U,injury_at_work_Y
0,1,2005,1,10,1,0,0,1,0
1,1,2005,1,13,0,1,0,1,0
2,1,2005,1,16,1,0,0,1,0
3,1,2005,1,11,0,1,0,1,0
4,1,2005,1,14,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
2718192,10,2015,2,10,1,0,0,1,0
2718194,12,2015,1,10,1,0,0,1,0
2718195,12,2015,1,7,0,1,0,1,0
2718196,7,2015,2,17,0,1,0,1,0


In [0]:
X = st.add_constant(X, prepend=False)
mdl = st.MNLogit(Y, X)
mdl_fit = mdl.fit(method='bfgs')
print (mdl_fit.summary())

/Users/Henry/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/Henry/miniconda3/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:2196: RuntimeWarning: invalid value encountered in true_divide
  return eXB/eXB.sum(1)[:,None]
/Users/Henry/miniconda3/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:2227: RuntimeWarning: divide by zero encountered in log
  logprob = np.log(self.cdf(np.dot(self.exog,params)))
/Users/Henry/miniconda3/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:2228: RuntimeWarning: invalid value encountered in multiply
  return np.sum(d * logprob)


KeyboardInterrupt: 